# Yaarl API Test Suite

In [1]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


LogBookDatabase - a dict of pandas logbooks, key is 'callsign' for both.
ContactDatabase - pandas dataframe
----> common interfaces
Logbook - an object.
LogEntry - an object.
Contact - an object
----> common interfaces

Databases:

- add
- remove
- amend
- get / set

- new / destroy / open / close

Objects:

- new
- amend
- get / set

Authentication:
- API key?
- ReadOnly mode for entire API - does not add the routes if enabled!

## API

- /[database]/[object] - method for operation

| Path | Method | Operation | Params |
|--|--|--|--|
| logbook/entry | GET | Return list of logbooks | (none) |
| logbook/entry | GET | Return logbook entries | logbook=[callsign], callsign=[search for callsign], etc |
| logbook/entry | POST | Add new entry | logbook=[callsign], callsign=[search for callsign], etc |
| logbook/entry | PUT | Update an entry  | logbook=[callsign], callsign=[search for callsign], etc |
| | | | |
| contact/contact | GET | Return list of contacts (shared between all) | (none) |
| contact/contact | GET | Return logbook entries | logbook=[callsign], callsign=[search for callsign], etc |
| logbook/entry | POST | Add new entry | logbook=[callsign], callsign=[search for callsign], etc |
| logbook/entry | PUT | Update an entry  | logbook=[callsign], callsign=[search for callsign], etc |






In [2]:
from config import config 

# Import Other Libraries
import pandas as pd

# Logbooks Database - a dict of dataframes
logbooks = {}

# Contacts Database
contacts = pd.DataFrame(columns=config['contact_cols'])

Create entry

In [3]:
data = [{'owner': 'dev_user','call_sign': 'Q12ABC','qth': 'Quarry City', 'first_name': 'Quinten', 'grid_square': 'QQ11QQ'}]
more_data = [{'owner': 'test_user','call_sign': 'Q12ABC','qth': 'Quarry City', 'first_name': 'Quinten'},
             {'owner': 'dev_user','call_sign': 'X34XYZ', 'first_name': 'Xavier'}
            ]
df = pd.DataFrame.from_dict(data, orient='columns')
df

,owner,call_sign,qth,first_name,grid_square
0,dev_user,Q12ABC,Quarry City,Quinten,QQ11QQ


Retrieve:

In [4]:
print (df.loc[df['owner'] == 'dev_user'])
print ()
_call_sign = 'X34XYZ'
_logged_in = 'dev_user'
df[(df['call_sign'] == _call_sign) & (df['owner'] == _logged_in) ].count()


      owner call_sign          qth first_name grid_square
0  dev_user    Q12ABC  Quarry City    Quinten      QQ11QQ



owner          0
call_sign      0
qth            0
first_name     0
grid_square    0
dtype: int64

In [17]:
from datetime import datetime
import pandas as pd
import tabulate

from config import config


# Class for a Contact

class Contact ():

  def __check_mandatory_fields (self, data):
    _missing = []
    for _field in config['contact_cols_mand']:
      if _field not in data.keys():
        _missing.append(_field)
    return _missing
  
  def __get_now_string (self):
    _dt = datetime.now()
    return str(_dt)

  # Init Function

  def __init__(self):
    self.data = {}
    for _key in config['contact_cols']:
       self.data[_key] = '' 
    self.data['updated'] = self.__get_date_time_now()
    
  def __init__(self, data):
    self.data = {}

    _missing = self.__check_mandatory_fields(data)
    # If we are not missing fields, go ahead
    if len(_missing) == 0:
      for _field in data.keys():
        if _field in config['contact_cols']:
          self.data[_field] = data[_field]
      self.data['updated'] = self.__get_now_string()
      self.fields = []

  # Print me...
  def __str__(self):
    _string = str(self.data)
    return _string
  
  # New --> Amend
  def new (self, data):
    return (self.amend(data))
  
  # Get
  def get (self):
    return self.data
  
  # Amend
  def amend (self, data):
     
    # Only copy over the fields we recognise from the schema
    # If we're not expecting it, bin it.  
    for _field in data.keys():
      if _field in config['contact_cols']:
        self.data[_field] = data[_field]
        
    self.data['updated'] = self.__get_now_string()
    self.fields = self.data.keys()


# Class for Contacts Database


class ContactsDB:
  
  # Init Function
  def __init__(self):
    self.contacts = pd.DataFrame(columns=config['contact_cols'])

  # Print me...
  def __str__(self):
    _string = str(contacts)
    return _string
  
  def schema (self):
    return config['contact_cols']
  
  def info (self):
    _string = str(self.contacts.shape) + "\n" + str(self.contacts.columns)
    return _string
  
  def add (self, data_dict):
    # works

    for _field in config['contact_cols_mand']:
      if _field not in data.keys():
        _missing.append(_field)
    return _missing

    print (f'--- Adding {data_dict}')
    print (f'Location index: {self.contacts.index}')
    self.contacts.loc[len(self.contacts)] = data_dict
    self.contacts.index = self.contacts.index + 1
    #self.contacts = self.contacts.append(data_dict, ignore_index=True)
    print (self.contacts.shape)

  def get (self, filter):
    # What about logged in user...?
    _result = self.contacts.loc[self.contacts['call_sign'] == filter['call_sign']]
    return _result
  
  def get_df (self):
    # What about logged in user...?

    return self.contacts


In [6]:
test_data = [
            {'owner': 'dev_user','call_sign': 'Q12ABC','qth': 'Quarry City', 'first_name': 'Quinten', 'grid_square': 'QQ11QQ'},
            {'owner': 'test_user','call_sign': 'Q12ABC','qth': 'Quarry City', 'first_name': 'Quinten'},
            {'owner': 'dev_user','call_sign': 'X34XYZ', 'first_name': 'Xavier'},
            {'owner': 'dev_user','call_sign': 'X34XYZ'},
            {'qth': 'Quarry City', 'first_name': 'Quinten', 'grid_square': 'QQ11QQ'}
        ]


In [18]:
print (f'Mandatory fields are: {config['contact_cols_mand']}')
#del cdb
cdb = ContactsDB()

for _d in test_data:
    _c = Contact(_d)
    cdb.add(_c)


Mandatory fields are: ['owner', 'call_sign', 'first_name']
--- Adding {'owner': 'dev_user', 'call_sign': 'Q12ABC', 'qth': 'Quarry City', 'first_name': 'Quinten', 'grid_square': 'QQ11QQ', 'updated': '2024-06-13 15:10:25.271546'}
Location index: RangeIndex(start=0, stop=0, step=1)
(1, 6)
--- Adding {'owner': 'test_user', 'call_sign': 'Q12ABC', 'qth': 'Quarry City', 'first_name': 'Quinten', 'updated': '2024-06-13 15:10:25.272679'}
Location index: Index([1], dtype='int64')
(1, 6)
--- Adding {'owner': 'dev_user', 'call_sign': 'X34XYZ', 'first_name': 'Xavier', 'updated': '2024-06-13 15:10:25.273240'}
Location index: Index([2], dtype='int64')
(2, 6)
--- Adding {}
Location index: Index([3, 2], dtype='int64')
(2, 6)
--- Adding {}
Location index: Index([4, 3], dtype='int64')
(3, 6)


In [9]:
#print (cdb)
_result = cdb.get({'call_sign': 'Q12ABC'})
print ()
print (_result)



Empty DataFrame
Columns: [owner, call_sign, qth, first_name, grid_square, updated]
Index: []


In [10]:
_df = cdb.get_df()
_df

,owner,call_sign,qth,first_name,grid_square,updated
4,"{'owner': 'dev_user', 'call_sign': 'Q12ABC', '...","{'owner': 'dev_user', 'call_sign': 'Q12ABC', '...","{'owner': 'dev_user', 'call_sign': 'Q12ABC', '...","{'owner': 'dev_user', 'call_sign': 'Q12ABC', '...","{'owner': 'dev_user', 'call_sign': 'Q12ABC', '...","{'owner': 'dev_user', 'call_sign': 'Q12ABC', '..."
3,"{'owner': 'test_user', 'call_sign': 'Q12ABC', ...","{'owner': 'test_user', 'call_sign': 'Q12ABC', ...","{'owner': 'test_user', 'call_sign': 'Q12ABC', ...","{'owner': 'test_user', 'call_sign': 'Q12ABC', ...","{'owner': 'test_user', 'call_sign': 'Q12ABC', ...","{'owner': 'test_user', 'call_sign': 'Q12ABC', ..."
2,"{'owner': 'dev_user', 'call_sign': 'X34XYZ', '...","{'owner': 'dev_user', 'call_sign': 'X34XYZ', '...","{'owner': 'dev_user', 'call_sign': 'X34XYZ', '...","{'owner': 'dev_user', 'call_sign': 'X34XYZ', '...","{'owner': 'dev_user', 'call_sign': 'X34XYZ', '...","{'owner': 'dev_user', 'call_sign': 'X34XYZ', '..."
1,{},{},{},{},{},{}
0,{},{},{},{},{},{}
